In [4]:
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver
import os
import time

#configure webdrive not to open windows
from selenium.webdriver.chrome.options import Options

TEMP_PATH = os.path.join('datasets', 'temperatures')


def fetch_daily_temp(page, date, temp_path=TEMP_PATH):
    '''Get daily temperatures from wunderground'''
    
    #configure webdrive not to open windows
    chrome_options = Options()
    chrome_options.add_argument('headless')
    
    #initialize driver
    # Change argument to the location you installed the chrome driver
    driver = webdriver.Chrome(r'C:\Users\rab53\OneDrive\Desktop\Python\chromedriver.exe', options=chrome_options)
    
    #add requested date to wunderground url
    URL = str(page) + str(date)
    driver.get(URL)
    time.sleep(4)
    r = driver.page_source
    driver.quit()
    
    #parse webpage
    soup = BS(r, 'html.parser')
    container = soup.find('lib-city-history-observation')
    check = container.find('tbody')
    
    #use pandas to get table
    try:
        df = pd.read_html(str(container))[0]
    except ValueError:
        bad_dates.append(date)
        
    if not os.path.isdir(temp_path):
        os.makedirs(temp_path)
        
    df.to_csv(os.path.join(temp_path, f'{date}.csv'))
    
    #pause again to prevent making too many requestions, robots.txt -> 3 seconds
    time.sleep(1)

In [5]:
dates_to_fetch = pd.period_range(start='1-13-2022', end='2-01-2022', freq='D')
dates_to_fetch

PeriodIndex(['2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16',
             '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20',
             '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24',
             '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28',
             '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01'],
            dtype='period[D]', freq='D')

In [6]:
web_page = 'https://www.wunderground.com/history/daily/us/tx/austin/KAUS/date/'

for i in dates_to_fetch:
    fetch_daily_temp(web_page, i)